In [1]:
import pandas as pd
import rltk
import re

wikicom = pd.read_csv("UScompanyWiki.csv", encoding='utf8')
jobscom = pd.read_csv("companyURL.csv", encoding='utf8')
# f = open("companyNameRev.txt", encoding='utf8')
# comList = [line.strip("\n").lower() for line in f.readlines()]
# f.close()

In [2]:
# remove "(United States)" from the company name

wikiList = [company.split(" (United States)")[0].lower() for company in wikicom['URILabel']]

In [3]:
wikicom['URILabel'] = wikiList

In [4]:
wikicom.sort_values(by= 'URILabel', axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last')
wikicom.reset_index(drop=True, inplace=True)

In [5]:
wikicom.fillna(-1, inplace=True)

In [6]:
comList = list(jobscom['Company Name'])
wikiList = list(wikicom['URILabel'])
comset = sorted(set(comList))
wikiset = sorted(set(wikiList))

In [7]:
simDic = dict()
for com1 in comset:
    for com2 in wikiset:
        if com1[0] == com2[0]:
            sim = rltk.jaro_winkler_similarity(com1, com2)
            if sim > 0.8:
                simDic.update({(com1, com2): sim})
        elif com1[0] < com2[0]:
            break

In [8]:
match = dict()
matchedKey = list()
for key in simDic:
    if simDic[key] == 1:
        key0 = key[0]
        match.update({key0:wikicom.iloc[wikiList.index(key0)][0]})
        matchedKey.append(key0)

In [9]:
new_dict = {key:val for key, val in simDic.items() if key[0] not in matchedKey and key[1] not in matchedKey} 

In [10]:
len(new_dict)

6165

In [11]:
def getCleanName (key):    
    newkey = re.sub(r" corporation", "", key)
    newkey = re.sub(r" incorporation", "", newkey)
    newkey = re.sub(r"[.,]", "",newkey)
    newkey = re.sub(r" llc", "", newkey)
    newkey = re.sub(r" inc", "", newkey)
    newkey = re.sub(r" corp", "", newkey)
    newkey = re.sub(r" co", "", newkey)
    newkey = re.sub(r"&", "", newkey)
    newkey = re.sub(r" and ", " ", newkey)
    newkey = re.sub(r"  ", " ", newkey)
    newkey = re.sub(r"technologies", "technology", newkey)
    newkey = re.sub(r"laboratories", "labs", newkey)    
    return newkey

In [12]:
matchedKey0 = list()
matchedKey1 = list()
for key in new_dict:
    key0 = key[0]
    key1 = key[1]
    key2 = getCleanName(key[0])
    key3 = getCleanName(key[1])
    sim = rltk.jaro_winkler_similarity(key2, key3)
    if sim == 1:
        match.update({key0: wikicom.iloc[wikiList.index(key1)][0]}) 
        matchedKey0.append(key0)
        matchedKey1.append(key1)

In [13]:
len(match)

174

In [14]:
new_dict = {key:val for key, val in new_dict.items() if key[0] not in matchedKey0 and key[1] not in matchedKey1} 

In [15]:
len(new_dict)

5825

In [16]:
def getURL (url):
    if url !=-1:
        newurl = re.sub(r".com", "", url)
        newurl = re.sub(r".org", "", url)
        newurl = re.sub(r"www.", "", newurl)
        newurl = re.sub(r"http:", "", newurl)
        newurl = re.sub(r"https:", "", newurl)
        newurl = re.sub(r"[:/.]", "", newurl)
        return newurl
    else:
        return -1

In [17]:
# website == comURI, and name similarity > 0.8
matchedKey0 = list()
matchedKey1 = list()
for key in new_dict:
    key0 = key[0]
    key1 = key[1]
    website = getURL(jobscom.iloc[comList.index(key0)][1])
    comURI = getURL(wikicom.iloc[wikiList.index(key1)][2])    
    if website == comURI and website !=-1:
        match.update({key0: wikicom.iloc[wikiList.index(key1)][0]}) 
        matchedKey0.append(key0)
        matchedKey1.append(key1)

In [18]:
len(match)

192

In [19]:
new_dict = {key:val for key, val in new_dict.items() if key[0] not in matchedKey0 and key[1] not in matchedKey1} 

In [20]:
len(new_dict)

5672

In [21]:
cnt = 0
for key, val in new_dict.items():
    if val >= 0.9:
        cnt +=1
        print(key, val)


('actioniq', 'actinix') 0.9214285714285715
('aditi consulting', 'aditi technologies') 0.901984126984127
('advanced food products', 'advanced powder solutions') 0.9036746411483253
('advantex', 'advanta') 0.9214285714285715
('advantex', 'advantagene') 0.9022727272727272
('aic', 'apic') 0.9249999999999999
('alk technologies', 'ada technologies') 0.9249999999999999
('alk technologies', 'altus technologies') 0.9211111111111111
('alk technologies', 'api technologies') 0.9249999999999999
('allied solutions', 'allied schools') 0.9047619047619048
('altran', 'adtran') 0.9
('altran', 'altera') 0.9222222222222223
('altran', 'amtran') 0.9
('american cybersystems', 'american systems') 0.9273809523809524
('amida technologies', 'ada technologies') 0.9666666666666666
('amida technologies', 'akamai technologies') 0.9341073271413829
('amida technologies', 'api technologies') 0.93125
('amida technologies', 'artium technologies') 0.900548245614035
('amida technologies', 'asius technologies') 0.9
('amrock',

In [22]:
cnt

222

('digitalocean', 'digital ocean'), ('peoplesbank', 'peoples bank'), ('nestle', 'nestlé'), 
('takeda pharmaceuticals', 'takeda pharmaceuticals north america')
('trilogy ed', 'trilogy')
('universal technologies (ny)', 'universal technology corporation')

key = matchedKey[12]
print(wikicom.iloc[wikiList.index(key)][0])
print(wikicom.iloc[wikiList.index(key)][1])
print(wikicom.iloc[wikiList.index(key)][2])
print(jobscom.iloc[comList.index(key)][0])
print(jobscom.iloc[comList.index(key)][1])

In [23]:
match

{'23andme': 'http://www.wikidata.org/entity/Q216272',
 'a place for mom': 'http://www.wikidata.org/entity/Q4658862',
 'abb': 'http://www.wikidata.org/entity/Q30338853',
 'acceleron pharma': 'http://www.wikidata.org/entity/Q16824238',
 'advanced bioscience laboratories': 'http://www.wikidata.org/entity/Q30253430',
 'aerovironment': 'http://www.wikidata.org/entity/Q919300',
 'agios pharmaceuticals': 'http://www.wikidata.org/entity/Q16824426',
 'alector': 'http://www.wikidata.org/entity/Q61934361',
 'alteryx': 'http://www.wikidata.org/entity/Q73883753',
 'amazon': 'http://www.wikidata.org/entity/Q3884',
 'analysis group': 'http://www.wikidata.org/entity/Q4751092',
 'analytical mechanics associates': 'http://www.wikidata.org/entity/Q41568369',
 'apellis pharmaceuticals': 'http://www.wikidata.org/entity/Q30286728',
 'applied materials': 'http://www.wikidata.org/entity/Q621610',
 'asml': 'http://www.wikidata.org/entity/Q30338395',
 'associated banc-corp': 'http://www.wikidata.org/entity/Q480

In [24]:
wikidata= pd.read_csv("wikidata.csv", encoding='utf8')

In [25]:
uris = [val for key, val in match.items()] 

In [26]:
rowList = list()
for i in range(len(wikidata)):
    if wikidata.iloc[i]['URI'] in uris:
        rowList.append(i)

In [27]:
len(rowList)

677

In [28]:
df = wikidata.iloc[rowList]

In [29]:
df
df.reset_index(drop=True, inplace=True)

In [30]:
import json
f = open("company-wikiurl.json", mode = "w+")
json.dump(match, f)
f.close()

In [32]:
df.to_csv(r'wikidata2.csv', index = False, header=True)

In [33]:
df

,URI,URILabel,comURI,founded_by,founded_byLabel,CEO,CEOLabel,parent,parentLabel,subsidiary,subsidiaryLabel,owned_by,owned_byLabel,owner_of,owner_ofLabel,total_revenue
0,http://www.wikidata.org/entity/Q216272,23andMe,https://www.23andme.com,http://www.wikidata.org/entity/Q2069573,Anne Wojcicki,http://www.wikidata.org/entity/Q2069573,Anne Wojcicki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://www.wikidata.org/entity/Q216272,23andMe,https://www.23andme.com,http://www.wikidata.org/entity/Q2069744,Linda Avey,http://www.wikidata.org/entity/Q2069573,Anne Wojcicki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://www.wikidata.org/entity/Q216272,23andMe,https://www.23andme.com,http://www.wikidata.org/entity/Q13377793,Paul Cusenza,http://www.wikidata.org/entity/Q2069573,Anne Wojcicki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://www.wikidata.org/entity/Q4658862,A Place for Mom,http://www.aplaceformom.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://www.wikidata.org/entity/Q30338853,ABB (United States),http://new.abb.com/us,NaN,NaN,NaN,NaN,http://www.wikidata.org/entity/Q52825,ABB Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,http://www.wikidata.org/entity/Q16824238,Acceleron Pharma,http://www.acceleron.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,http://www.wikidata.org/entity/Q30253430,Advanced Bioscience Laboratories,http://www.ablinc.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,http://www.wikidata.org/entity/Q919300,AeroVironment,http://www.avinc.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,http://www.wikidata.org/entity/Q16824426,Agios Pharmaceuticals,http://www.agios.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,http://www.wikidata.org/entity/Q61934361,Alector (United States),https://www.alector.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
